# Bug1
## Equipo 3

Primero se importan las librerias necesarias.

In [1]:
import rospy
import numpy as np
from geometry_msgs.msg import Twist, PoseStamped
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from smach import State, StateMachine
import smach_ros
from tf.transformations import euler_from_quaternion

Controlador simple

In [2]:
class ControlPID:
    def __init__(self, rate, kp = 1, ki = 0, kd = 0):
        self.rate = rate
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.i = 0
        self.ea = 0
        self.pred = 0

    def update(self, current, goal):
        ep = goal - current
        self.i = self.i + self.ki*ep*self.rate
        ed = (ep - self.ea)/self.rate
        self.ea = ep
        self.pred = current + ep*self.kp + self.i + ed*self.kd


Luego se declaran variables globales.

In [3]:
pos = Twist()
c_vel = Twist()
pub = rospy.Publisher('/cmd_vel',Twist,queue_size=10)
rospy.init_node('bug1')
rate_dur = 10
rate = rospy.Rate(rate_dur)
rate_dur = 1/rate_dur
puntos = []
goal = []

nfb = False
ilb = False
sl = 0
sor = 1
temp_start = []
temp_goal = []
dis_r = 0
dis_l = 0
wsd = False
turnPID = ControlPID(rate_dur,kp = 4)
fowaPID = ControlPID(rate_dur,kp = 1)

Declaramos el callback de Odometría el cual se encarga de dar la posición del robot de acuerdo a sus cálculos de las velocidades hechas en cada motor.

In [4]:
def callbackOdom(msg):
    global pos
    global c_vel
    pos.linear.x = msg.pose.pose.position.x
    pos.linear.y = msg.pose.pose.position.y
    rot_q = msg.pose.pose.orientation
    (_,_,pos.angular.z) = euler_from_quaternion([rot_q.x, rot_q.y, rot_q.z, rot_q.w])
    c_vel.linear.x = np.sqrt(msg.twist.twist.linear.x**2 + msg.twist.twist.linear.y**2)
    c_vel.angular.z = msg.twist.twist.angular.z

Esta función sirve para dar la diferencia de ángulo entre dos ángulos en el plano cartesiano.

In [5]:
def smallest_angle_diff(t,s):
    a = t - s
    a -= 2*np.pi if a > np.pi else -2*np.pi if a < -np.pi else 0
    return a

Función para medir distancia entre dos puntos

In [6]:
def dis(src, goal):
    return np.sqrt((goal[0]-src[0])**2 + (goal[1]-src[1])**2)

Este callback verifica con el LiDAR los obstaculos más cercanos los cuales si se detecta alguno calcula la cantidad de freno y desviación para evitar colisión.

In [7]:
def callbackScan(msg):
    global nfb
    global ilb
    global sl

    #Get angles array
    scan = np.array(msg.ranges)
    ang = msg.angle_min - np.pi
    andf = msg.angle_max - np.pi
    inc = msg.angle_increment
    angles = np.arange(ang,andf+inc,inc)
    
    #Get only front detected points
    scan_f = scan[np.r_[-45:0]]
    scan_f = scan_f[~np.isnan(scan_f)]
    scan_f = scan_f[np.isfinite(scan_f)]

    #Get only left detected points
    anglesl = angles[np.r_[45:90]]
    scan_l = scan[np.r_[45:90]]
    anglesl = anglesl[~np.isnan(scan_l)]
    scan_l = scan_l[~np.isnan(scan_l)]
    anglesl = anglesl[np.isfinite(scan_l)]
    scan_l = scan_l[np.isfinite(scan_l)]

    #Get booleans if obstacle in front and wall in its left
    nfb = not scan_f[scan_f < 0.45].shape[0] > 0 and not scan_l[scan_l < 0.3].shape[0] > 0
    ilb = scan_l[scan_l < 0.45].shape[0] > 0

    #Calculate slope with regression and give it as error
    if scan_l[scan_l < 0.6].shape[0] >= 13:
        px = scan_l*np.cos(anglesl)
        py = scan_l*np.sin(anglesl)
        sl = np.sum((px - np.average(px))*(py -np.average(py)))/np.sum((px - np.average(px))**2)

Este callback recibe nuevos puntos objetivo

In [8]:
def callbackGoalPoint(msg):
    global puntos
    puntos.append([msg.pose.position.x, msg.pose.position.y])

Este es el primer estado el cual envía al robot la velocidad de giro necesaria para alcanzar el ángulo mínimo entre el turtlebot y el punto objetivo, y así iniciar haciendo un recorrido en línea recta.

In [9]:
class Turn(State):
    def __init__(self):
        State.__init__(self, outcomes=['turnr','fowardr','wait'])
        
    def execute(self, ud):
        global pos
        global c_vel
        global pub
        global rate
        global puntos
        global turnPID
        global goal

        if len(puntos) == 0:
            return 'wait'
        goal = puntos[0]
        vel = Twist()

        c_ang = np.arctan2(goal[1]-pos.linear.y,goal[0]-pos.linear.x)
        o_ang = smallest_angle_diff(c_ang,pos.angular.z)
        o_ang = o_ang if abs(o_ang) <= np.pi/4 else (np.pi/4)*np.sign(o_ang)
        turnPID.update(c_vel.angular.z, o_ang)
        vel.angular.z = turnPID.pred
        vel.angular.z = vel.angular.z if abs(vel.angular.z) <= np.pi/4 else (np.pi/4)*np.sign(vel.angular.z)
        pub.publish(vel)
        rate.sleep()

        if abs(smallest_angle_diff(c_ang,pos.angular.z)) > 0.05:
            return 'turnr'
        else:
            vel.angular.z = 0
            pub.publish(vel)
            rate.sleep()
            pub.publish(vel)
            rate.sleep()
            return 'fowardr'

El segundo estado consiste en avanzar y corregir el rumbo si se está desviando a partir de un giro muy discreto. Si hay obstaculo aquí se aplica el freno y la desviación.

In [10]:
class Advance(State):
    def __init__(self):
        State.__init__(self, outcomes=['turnr','fowardr','wait','at'])
        
    def execute(self, ud):
        global pos
        global pub
        global rate
        global puntos

        global turnPID
        global fowaPID

        global nfb
        global temp_start
        global temp_goal

        goal = puntos[0]
        vel = Twist()

        c_ang = np.arctan2(goal[1]-pos.linear.y,goal[0]-pos.linear.x)
        o_ang = smallest_angle_diff(c_ang,pos.angular.z)
        o_ang = o_ang if abs(o_ang) <= np.pi/4 else (np.pi/4)*np.sign(o_ang)
        turnPID.update(c_vel.angular.z, o_ang)
        vel.angular.z = turnPID.pred
        vel.angular.z = vel.angular.z if abs(vel.angular.z) <= np.pi/4 else (np.pi/4)*np.sign(vel.angular.z)

        c_dis = dis([pos.linear.x,pos.linear.y],goal)
        o_dis = c_dis if abs(c_dis) <= 0.2 else 0.2*np.sign(c_dis)
        fowaPID.update(c_vel.linear.x, o_dis)
        vel.linear.x = fowaPID.pred
        vel.linear.x = vel.linear.x if abs(vel.linear.x) <= 0.2 else 0.2*np.sign(vel.linear.x)
        pub.publish(vel)
        rate.sleep()
        
        if nfb:
            if c_dis > 0.05:
                return 'fowardr'
            else:
                puntos.pop(0)
                if len(puntos) == 0:
                    return 'wait'
                else:
                    return 'turnr'
        else:
            temp_start = [pos.linear.x,pos.linear.y]
            temp_goal = [pos.linear.x,pos.linear.y]
            return 'at'

El tercer estado consiste de esperar nuevos puntos objetivo

In [11]:
class Wait(State):
    def __init__(self):
        State.__init__(self, outcomes=['turnr','wait'])
    def execute(self, ud):
        global pub
        global rate
        vel = Twist()
        vel.angular.z = 0
        vel.linear.x = 0
        pub.publish(vel)
        rate.sleep()
        pub.publish(vel)
        rate.sleep()
        if (len(puntos)) > 0:
            return 'turnr'
        else:
            return 'wait'

El estado de giro solo se activa cuando tiene obstaculo enfrente.

In [12]:
class Turn_so(State):
    def __init__(self):
        State.__init__(self, outcomes=['a','at','t','turnr'])
    def execute(self, ud):
        global pub
        global rate
        global rate_dur
        global nfb
        global ilb
        global c_vel

        global wsd
        global temp_goal
        global goal
        global dis_l
        global dis_r
        global sor
        
        vel = Twist()
        turnPID.update(c_vel.angular.z, -np.pi/4*sor)
        vel.angular.z = turnPID.pred
        pub.publish(vel)
        rate.sleep()

        if not wsd:
            t_temp_goal = [pos.linear.x,pos.linear.y]
            temp_goal = t_temp_goal if dis(t_temp_goal,goal) < dis(temp_goal,goal) else temp_goal
            dis_r += c_vel.linear.x*rate_dur + dis_l if t_temp_goal == temp_goal else 0
            dis_l = 0 if t_temp_goal == temp_goal else dis_l
            dis_l += c_vel.linear.x*rate_dur if t_temp_goal != temp_goal else 0
            print(dis_l, dis_r,c_vel.linear.x*rate_dur)
            if dis(t_temp_goal,temp_start) < 0.2 and (dis_l + dis_r) > 1:
                wsd = True
        else:
            if dis_l > dis_r:
                sor = -1
            else:
                sor = 1
            if dis([pos.linear.x,pos.linear.y],temp_goal) < 0.1:
                temp_goal = []
                dis_l = 0
                dis_r = 0
                wsd = False
                return 'turnr'
            
        if not nfb:
            return "t"
        elif ilb:
            return "a"
        else:
            return "at"

El estado de avance y giro se activa cuando no hay obstaculos enfrente ni a su izquierda.

In [13]:
class AdvanceTurn(State):
    def __init__(self):
        State.__init__(self, outcomes=['a','at','t','turnr'])
    def execute(self, ud):
        global pub
        global rate
        global nfb
        global ilb
        global c_vel

        global wsd
        global temp_goal
        global goal
        global dis_l
        global dis_r
        global sor
        
        vel = Twist()
        turnPID.update(c_vel.angular.z, np.pi/4)
        vel.angular.z = turnPID.pred
        fowaPID.update(c_vel.linear.x, 0.1)
        vel.linear.x = fowaPID.pred
        print(vel.angular.z)
        pub.publish(vel)
        rate.sleep()

        if not wsd:
            t_temp_goal = [pos.linear.x,pos.linear.y]
            temp_goal = t_temp_goal if dis(t_temp_goal,goal) < dis(temp_goal,goal) else temp_goal
            dis_r += c_vel.linear.x*rate_dur + dis_l if t_temp_goal == temp_goal else 0
            dis_l = 0 if t_temp_goal == temp_goal else dis_l
            dis_l += c_vel.linear.x*rate_dur if t_temp_goal != temp_goal else 0
            print(dis_l, dis_r,c_vel.linear.x*rate_dur)
            if dis(t_temp_goal,temp_start) < 0.2 and (dis_l + dis_r) > 1:
                wsd = True
        else:
            if dis_l > dis_r:
                sor = -1
            else:
                sor = 1
            if dis([pos.linear.x,pos.linear.y],temp_goal) < 0.1:
                temp_goal = []
                dis_l = 0
                dis_r = 0
                wsd = False
                return 'turnr'
            
        if not nfb:
            return "t"
        elif ilb:
            return "a"
        else:
            return "at"

El estado de avance se activa cuando no hay obstáculo en frente y tiene pared a su izquierda. Para estar paralelo a la pared utiliza la pendiente como error donde 0 sería que está totalmente paralelo.

In [14]:
class Advance_so(State):
    def __init__(self):
        State.__init__(self, outcomes=['a','at','t','turnr'])
    def execute(self, ud):
        global pub
        global rate
        global nfb
        global ilb
        global sl

        global wsd
        global temp_goal
        global goal
        global dis_l
        global dis_r
        global sor
        
        vel = Twist()
        turnPID.update(c_vel.angular.z, sl)
        vel.angular.z = turnPID.pred
        vel.angular.z = vel.angular.z if abs(vel.angular.z) <= np.pi/4 else (np.pi/4)*np.sign(vel.angular.z)
        fowaPID.update(c_vel.linear.x, 0.2)
        vel.linear.x = fowaPID.pred
        pub.publish(vel)
        rate.sleep()

        if not wsd:
            t_temp_goal = [pos.linear.x,pos.linear.y]
            temp_goal = t_temp_goal if dis(t_temp_goal,goal) < dis(temp_goal,goal) else temp_goal
            dis_r += c_vel.linear.x*rate_dur + dis_l if t_temp_goal == temp_goal else 0
            dis_l = 0 if t_temp_goal == temp_goal else dis_l
            dis_l += c_vel.linear.x*rate_dur if t_temp_goal != temp_goal else 0
            print(dis_l, dis_r,c_vel.linear.x*rate_dur)
            if dis(t_temp_goal,temp_start) < 0.2 and (dis_l + dis_r) > 1:
                wsd = True
        else:
            if dis_l > dis_r:
                sor = -1
            else:
                sor = 1
            if dis([pos.linear.x,pos.linear.y],temp_goal) < 0.1:
                temp_goal = []
                dis_l = 0
                dis_r = 0
                wsd = False
                return 'turnr'

        if not nfb:
            return "t"
        elif ilb:
            return "a"
        else:
            return "at"

Ésta es la función principal donde se inicializan los nodos suscriptores y la máquina de estados.

In [15]:
def main():
    
    odom = rospy.Subscriber('/odom_gazebo',Odometry,callbackOdom)
    scanS = rospy.Subscriber('/scan',LaserScan,callbackScan)
    Gpoints = rospy.Subscriber('/move_base_simple/goal',PoseStamped,callbackGoalPoint)
    
    sm = StateMachine(outcomes=['succeeded'])
    sm.userdata.sm_input = 0

    with sm:

        StateMachine.add('WAIT', Wait(), transitions={'turnr':'TURN','wait':'WAIT'})
        StateMachine.add('TURN', Turn(), transitions={'turnr':'TURN','fowardr':'FOWARD','wait':'WAIT'})
        StateMachine.add('FOWARD', Advance(), transitions={'turnr':'TURN','fowardr':'FOWARD','wait':'WAIT','at':'ADVANCETURN'})
        StateMachine.add('TURN_so', Turn_so(), transitions={'t':'TURN_so','at':'ADVANCETURN','a':'ADVANCE_so','turnr':'TURN'})
        StateMachine.add('ADVANCE_so', Advance_so(), transitions={'t':'TURN_so','at':'ADVANCETURN','a':'ADVANCE_so','turnr':'TURN'})
        StateMachine.add('ADVANCETURN', AdvanceTurn(), transitions={'t':'TURN_so','at':'ADVANCETURN','a':'ADVANCE_so','turnr':'TURN'})
    
    sis = smach_ros.IntrospectionServer('server_name', sm, '/SM_ROOT')
    rospy.sleep(1)
    sis.start()
    
    outcome = sm.execute()

    sis.stop()
    
    rospy.sleep(1)

Finalmente, para ejecutar el código oficialmente (sin antes haber ya ejecutado las celdas anteriores y lanzado el mundo de turtlebot en Gazebo), se ejecuta la siguiente celda para iniciar con el proceso:

In [16]:
if __name__ == '__main__':
    try:
        main()
    except rospy.ROSInterruptException:
        pass

[INFO] [1683400821.151887, 70.672000]: State machine starting in initial state 'WAIT' with userdata: 
	['sm_input']
[INFO] [1683400821.252803, 70.774000]: State machine transitioning 'WAIT':'wait'-->'WAIT'
[INFO] [1683400821.453644, 70.974000]: State machine transitioning 'WAIT':'wait'-->'WAIT'
[INFO] [1683400821.654254, 71.174000]: State machine transitioning 'WAIT':'wait'-->'WAIT'
[INFO] [1683400821.856240, 71.374000]: State machine transitioning 'WAIT':'wait'-->'WAIT'
[INFO] [1683400822.056151, 71.574000]: State machine transitioning 'WAIT':'wait'-->'WAIT'
[INFO] [1683400822.258460, 71.774000]: State machine transitioning 'WAIT':'wait'-->'WAIT'
[INFO] [1683400822.458032, 71.974000]: State machine transitioning 'WAIT':'wait'-->'WAIT'
[INFO] [1683400822.659336, 72.174000]: State machine transitioning 'WAIT':'wait'-->'WAIT'
[INFO] [1683400822.859860, 72.374000]: State machine transitioning 'WAIT':'wait'-->'WAIT'
[INFO] [1683400823.060109, 72.574000]: State machine transitioning 'WAIT':

[ERROR] [1683400945.309806, 194.475000]: InvalidUserCodeError: Could not execute state 'WAIT' of type '<__main__.Wait object at 0x7fe9244bd5e0>': Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/smach/state_machine.py", line 250, in _update_once
    outcome = self._current_state.execute(
  File "/tmp/ipykernel_4534/2759226230.py", line 13, in execute
    rate.sleep()
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/timer.py", line 103, in sleep
    sleep(self._remaining(curr_time))
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/timer.py", line 159, in sleep
    rostime_cond.wait(0.5)
  File "/usr/lib/python3.8/threading.py", line 306, in wait
    gotit = waiter.acquire(True, timeout)
KeyboardInterrupt

/opt/ros/noetic/lib/python3/dist-packages/smach/state_machine.py:76: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return {k:v for (k, v) in self.__dict__.items() if k is not "_state_transitioning_lock"}
/opt/ros/noetic/l

InvalidUserCodeError: Could not execute state 'WAIT' of type '<__main__.Wait object at 0x7fe9244bd5e0>': Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/smach/state_machine.py", line 250, in _update_once
    outcome = self._current_state.execute(
  File "/tmp/ipykernel_4534/2759226230.py", line 13, in execute
    rate.sleep()
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/timer.py", line 103, in sleep
    sleep(self._remaining(curr_time))
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/timer.py", line 159, in sleep
    rostime_cond.wait(0.5)
  File "/usr/lib/python3.8/threading.py", line 306, in wait
    gotit = waiter.acquire(True, timeout)
KeyboardInterrupt
